In [4]:
from models.ts.cycles import forecast_serials, trade_serials

currency = "GBPJPY"
timeframe_m = 15
lags = 8

df, forecast_df = forecast_serials(currency, lags, timeframe_m)
df.tail(lags)

,close
time,
2024-12-05 20:15:00,191.279
2024-12-05 20:30:00,191.324
2024-12-05 20:45:00,191.354
2024-12-05 21:00:00,191.411
2024-12-05 21:15:00,191.366
2024-12-05 21:30:00,191.454
2024-12-05 21:45:00,191.300
2024-12-05 22:00:00,191.332


In [2]:
forecast_df

,predicted_close
time,
2024-12-05 22:15:00,191.279190
2024-12-05 22:30:00,191.319519
2024-12-05 22:45:00,191.375046
2024-12-05 23:00:00,191.422638
2024-12-05 23:15:00,191.369125
2024-12-05 23:30:00,191.435013
2024-12-05 23:45:00,191.299942
2024-12-06 00:00:00,191.285385


In [3]:
trade_serials(currency, df, forecast_df)


.....................................REQUEST:..........................................

predicted_close == current_close:

current_price   => 191.285
predicted_close => 191.285
target_profit   => 0

.....................................RESPONSE:..........................................

.....................I DON'T KNOW HOW TO HELP YOU!.....................
